In [1]:
#dependency install
# %pip install pandas
# %pip install openpyxl

In [25]:
#imports
import csv
import pandas as pd
import hw0
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy

In [3]:
NAN = float("nan")

### Reading the data

In [4]:
# #read CSV
# rawDataHeader = []
# rawData = []
# with open("eurovision_song_contest_1957_2023.csv") as csvfile:
#     reader = csv.reader(csvfile, delimiter=",")
#     rawDataHeader = next(reader)
#     for row in reader:
#         rawData.append(row)

In [5]:
pathOriginalDataFile = "data/eurovision_song_contest_1957_2023.xlsx"
origDataFrame = pd.read_excel(pathOriginalDataFile)

In [6]:
origDataFrame.keys()[6]

'Points      '

In [7]:
#fixing this bug-inducing column/attribute name
oldkey = origDataFrame.keys()[6]
newkey = origDataFrame.keys()[6].rstrip()
print("\"{}\"".format(oldkey))
print("\"{}\"".format(newkey))
origDataFrame.rename(columns={oldkey : newkey}, inplace=True)

"Points      "
"Points"


In [8]:
origDataFrame.keys()[6]

'Points'

## Version 1

### Data preparation

Poskusimo tako, da oblikujemo tabelo, kjer so vrstice glasujoče države, stolpci pa države, za katere so glasovale. V celicah so števila točk glasovanja. Stolpci z glasovi so za vsako leto posebej, leta dodamo tabeli (širimo v desno).

In [9]:
#drop entries that aren't for finals
prep1orig = origDataFrame[origDataFrame["(semi-) final"] == "f"]

In [10]:
# to je kr neki, ne dela - pridejo razlicno dolgi vektorji

# #extract uniques
# prep1years = prep1orig["Year"].unique()
# prep1voters = prep1orig["From country"].unique()
# prep1votedFor = prep1orig["To country"].unique()

# #prepare the data structure
# prep1data = {}
# for voterCountry in prep1voters:
#     prep1data[voterCountry] = {"keys" : [], "votes" : []}

# #fill the structure with data
# for year in prep1years:
#     yearSelection = prep1orig[prep1orig["Year"] == year]
#     for voterCountry in prep1voters:
#         voterSelection = yearSelection[yearSelection["From country"] == voterCountry]
#         for row in voterSelection.iterrows():
#             prep1data[voterCountry]["keys"].append((row[1]["To country"], year))
#             prep1data[voterCountry]["votes"].append(row[1]["Points"])

In [11]:
# for item in prep1data.items():
#     print(item[0])
#     print(len(item[1]["keys"]))
#     print(len(item[1]["votes"]))
#     print()

In [20]:
prep1years = prep1orig["Year"].unique()
prep1voters = prep1orig["From country"].unique()
prep1votedFor = prep1orig["To country"].unique()

prep1data = {}
for i in range(len(prep1voters)):
    voter = prep1voters[i]
    prep1data[voter] = []

for year in prep1years:
    yearSelection = prep1orig[prep1orig["Year"] == year]
    for voter in prep1voters:
        voterSelection = yearSelection[yearSelection["From country"] == voter]
        newVotes = []
        for votedFor in prep1votedFor:
            lineSelection = voterSelection[voterSelection["To country"] == votedFor]
            newVoteVal = NAN
            if len(lineSelection["Points"].values) > 0:
                newVoteVal = lineSelection["Points"].values[0]
            newVotes.append(newVoteVal)
        prep1data[voter] += newVotes

### Clustering

In [21]:
#params
distance1 = hw0.euclidean_dist
linkage1 = hw0.single_linkage
distFun1 = lambda c1, c2: linkage1(c1, c2, distance1)

In [22]:
hc1 = hw0.HierarchicalClustering(distFun1)
clusters1 = hc1.run(prep1data)

In [23]:
clusters1

[[[[[[[[[[[[[[[['Lithuania'], ['Latvia']], ['Estonia']], ['Ukraine']],
            ['Poland']],
           ['Belarus']],
          [[[[[[[[[[[[[[[[[[[[[['San Marino'], ['Rest of the World']],
                              ['Romania']],
                             ['Azerbaijan']],
                            ['Iceland']],
                           ['Cyprus']],
                          ['Malta']],
                         ['Australia']],
                        ['France']],
                       ['Greece']],
                      [[[[[[[[[[[[[[['Monaco'], ['Yugoslavia']],
                                   ['Luxembourg']],
                                  ['Spain']],
                                 ['Austria']],
                                ['Norway']],
                               ['Netherlands']],
                              ['United Kingdom']],
                             ['Italy']],
                            ['Germany']],
                           ['Sweden']],
      

### Visualization